# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Us,49.100945,1.967247,-0.84,100,34,1.03,FR,1741056538
1,1,Uz,42.966369,-0.085232,4.81,85,97,3.23,FR,1741056539
2,2,City of Osh,40.536190,72.803362,-5.55,68,16,3.00,KG,1741056540
3,3,Юськи,56.660308,53.080233,-6.80,96,100,3.77,RU,1741056541
4,4,Geva,32.566336,35.371549,12.80,56,100,1.94,IL,1741056543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Create an interactive map using hvplot
city_map_hvplot = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",  # Scale point size based on humidity
    color="Humidity",
    cmap="Blues",
    alpha=0.7,
    frame_width=700,
    frame_height=500,
    title="City Locations with Humidity Levels"
)

# Display the interactive map
city_map_hvplot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Define criteria to narrow down cities (example criteria)
# You can adjust these conditions based on your specific requirements
criteria_df = city_data_df[
    (city_data_df["Max Temp"] > 0) &  # Cities with above freezing temperatures
    (city_data_df["Humidity"] < 80) &  # Cities with humidity below 80%
    (city_data_df["Wind Speed"] < 10)  # Cities with wind speed below 10 m/s
]

# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Geva,32.566336,35.371549,12.80,56,100,1.94,IL,1741056543
6,6,Innermedøya,66.878996,13.446508,2.23,62,94,4.45,NO,1741056546
7,7,Ip,47.229321,22.606204,1.18,78,8,0.93,RO,1741056547
13,13,California,25.203072,55.143679,18.02,53,32,6.14,AE,1741056555
17,17,Postau,48.648269,12.326652,0.57,73,100,1.48,DE,1741056560
...,...,...,...,...,...,...,...,...,...,...
139,139,Chandigarh,30.733442,76.779714,17.24,68,69,2.37,IN,1741056723
148,148,Grove,36.589862,-94.768508,15.08,72,75,7.20,US,1741056734
155,155,Makaprr,8.772887,-11.945153,22.25,63,43,1.36,SL,1741056743
156,156,Makolor,8.759026,-11.955369,22.27,63,43,1.41,SL,1741056745


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create a new DataFrame for hotel data
hotel_df = criteria_df.copy()

# Keep only relevant columns
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column for "Hotel Name" to store results from Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
4,Geva,IL,32.566336,35.371549,56,
6,Innermedøya,NO,66.878996,13.446508,62,
7,Ip,RO,47.229321,22.606204,78,
13,California,AE,25.203072,55.143679,53,
17,Postau,DE,48.648269,12.326652,73,
...,...,...,...,...,...,...
139,Chandigarh,IN,30.733442,76.779714,68,
148,Grove,US,36.589862,-94.768508,72,
155,Makaprr,SL,8.772887,-11.945153,63,
156,Makolor,SL,8.759026,-11.955369,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
import requests

# Define search radius in meters (e.g., 5000m = 5km)
radius = 5000  

# Base URL for Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

# API key (replace with your actual key)
geoapify_key = "486fbdf0e14a46f6b208a317e7467555"

# Set parameters to search for hotels
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Fetch only the closest hotel
    "apiKey": geoapify_key
}

# Print a message to follow up on the hotel search
print("Starting hotel search...\n")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - Nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search...

Geva - Nearest hotel: Ein Harod Country Sites and Guesthouse
Innermedøya - Nearest hotel: No hotel found
Ip - Nearest hotel: No hotel found
California - Nearest hotel: لبنان
Postau - Nearest hotel: Hotel Wörth
Fr - Nearest hotel: No hotel found
فر - Nearest hotel: No hotel found
Far - Nearest hotel: No hotel found
Deh - Nearest hotel: No hotel found
De - Nearest hotel: No hotel found
Chun - Nearest hotel: No hotel found
Chanthaburi - Nearest hotel: Chailee Hotel
Rovello Porro - Nearest hotel: No hotel found
Ru - Nearest hotel: Albergo Ristorante Lusiana
Bar - Nearest hotel: Agape
Bridge - Nearest hotel: Ebury Hotel
Mexico - Nearest hotel: لبنان
Tokyo - Nearest hotel: パシフィックセンチュリープレイス丸の内
JP Nagar - Nearest hotel: Emilion HHI Select
JP Walkers Addition - Nearest hotel: Marriott
Junction City - Nearest hotel: No hotel found
Harre - Nearest hotel: Ranch Don Diego
Blue Eye - Nearest hotel: No hotel found
Zone 4 - Nearest hotel: Residence Inn
Turru - Nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
4,Geva,IL,32.566336,35.371549,56,Ein Harod Country Sites and Guesthouse
6,Innermedøya,NO,66.878996,13.446508,62,No hotel found
7,Ip,RO,47.229321,22.606204,78,No hotel found
13,California,AE,25.203072,55.143679,53,لبنان
17,Postau,DE,48.648269,12.326652,73,Hotel Wörth
...,...,...,...,...,...,...
139,Chandigarh,IN,30.733442,76.779714,68,Hotel Sunbeam
148,Grove,US,36.589862,-94.768508,72,Best Western Timberridge Inn
155,Makaprr,SL,8.772887,-11.945153,63,No hotel found
156,Makolor,SL,8.759026,-11.955369,63,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
# Drop rows where "Hotel Name" is "No hotel found"
hotel_df_filtered = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

In [15]:
%%capture --no-display

# Create a hover message with city, hotel, and country info
hotel_df_filtered["Hover Info"] = hotel_df_filtered.apply(
    lambda row: f"City: {row['City']}<br>Hotel: {row['Hotel Name']}<br>Country: {row['Country']}",
    axis=1
)

# Create an interactive map using hvplot with only cities that have hotels
hotel_map_hvplot_filtered = hotel_df_filtered.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",  # Scale point size based on humidity
    color="Humidity",
    cmap="Blues",
    alpha=0.7,
    hover_cols=["Hover Info"],  # Display city, hotel, and country info on hover
    frame_width=700,
    frame_height=500,
    title="Interactive Map of Cities with Hotels"
)

# Display the interactive map
hotel_map_hvplot_filtered


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hover Info)